In [1]:
import pandas as pd

In [2]:
pd.read_csv("../exp1/interpol_baselines/spatial_gp_rmse.csv").mean(axis = 0).mean()

15.6242649228566

### TO DO
* Kriging and IDW
* Sequential Code
* No Notion of Last K Days

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

In [7]:
# from polire.interpolate import  Kriging, Idw
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [8]:
df = pd.read_csv("../data/beijinga_scaled.csv", index_col = 0)

In [9]:
df.head()

,latitude,longitude,ts,station_id,PM2.5
0,0.58292,0.17557,0.000000,1001.0,156.250000
0,0.58292,0.17557,0.002747,1001.0,24.833333
0,0.58292,0.17557,0.005495,1001.0,72.583333
0,0.58292,0.17557,0.008242,1001.0,30.043478
0,0.58292,0.17557,0.010989,1001.0,23.083333


In [10]:
# for i in range(10):
df['PM2.5'].describe()

count    12542.000000
mean        85.948717
std         70.715368
min          3.000000
25%         33.958333
50%         67.362500
75%        116.199728
max        590.083333
Name: PM2.5, dtype: float64

In [11]:
import matplotlib.pyplot as plt
plt.hist(df['PM2.5'])

(array([5.754e+03, 3.851e+03, 1.706e+03, 6.820e+02, 3.020e+02, 1.690e+02,
        4.800e+01, 2.000e+01, 9.000e+00, 1.000e+00]),
 array([  3.        ,  61.70833333, 120.41666667, 179.125     ,
        237.83333333, 296.54166667, 355.25      , 413.95833333,
        472.66666667, 531.375     , 590.08333333]),
 <a list of 10 Patch objects>)

In [12]:
timestamps = df['ts'].unique()
timestamps.sort()
stations = df['station_id'].unique()

In [13]:
stations.sort()

In [14]:
kf = KFold(n_splits=6, random_state=0, shuffle=True)

In [15]:
df.head()

,latitude,longitude,ts,station_id,PM2.5
0,0.58292,0.17557,0.000000,1001.0,156.250000
0,0.58292,0.17557,0.002747,1001.0,24.833333
0,0.58292,0.17557,0.005495,1001.0,72.583333
0,0.58292,0.17557,0.008242,1001.0,30.043478
0,0.58292,0.17557,0.010989,1001.0,23.083333


In [16]:
for train_index, test_index in kf.split(stations):
    print(train_index, test_index)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 17 18 19 21 23 24 25 26 27
 28 29 32 33 34 35] [15 16 20 22 30 31]
[ 0  1  3  4  5  6  7  8  9 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 28 30 31 32 33 34] [ 2 10 11 27 29 35]
[ 0  1  2  3  4  6  7  9 10 11 12 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 29 30 31 34 35] [ 5  8 13 28 32 33]
[ 0  2  3  4  5  6  8  9 10 11 13 15 16 18 19 20 21 22 23 24 25 27 28 29
 30 31 32 33 34 35] [ 1  7 12 14 17 26]
[ 0  1  2  3  5  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22 26 27 28 29
 30 31 32 33 34 35] [ 4  6 18 23 24 25]
[ 1  2  4  5  6  7  8 10 11 12 13 14 15 16 17 18 20 22 23 24 25 26 27 28
 29 30 31 32 33 35] [ 0  3  9 19 21 34]


In [17]:
from sklearn.neighbors import KNeighborsRegressor

In [18]:
import gpflow
# /import tensorflow as tf

W1102 12:02:19.332079 139634146854656 deprecation_wrapper.py:119] From /data/anaconda/envs/py36/lib/python3.6/site-packages/gpflow/session_manager.py:31: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1102 12:02:19.333387 139634146854656 deprecation_wrapper.py:119] From /data/anaconda/envs/py36/lib/python3.6/site-packages/gpflow/misc.py:27: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W1102 12:02:19.472523 139634146854656 deprecation_wrapper.py:119] From /data/anaconda/envs/py36/lib/python3.6/site-packages/gpflow/training/tensorflow_optimizer.py:169: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W1102 12:02:19.473452 139634146854656 deprecation_wrapper.py:119] From /data/anaconda/envs/py36/lib/python3.6/site-packages/gpflow/training/tensorflow_optimizer.py:156: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1102

In [19]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)


In [24]:
rmse = {'K':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}, 'I':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},'KNN':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},
       'GP':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}}
mae = {'K':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}, 'I':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},'KNN':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},
      'GP':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}}


for timestamp in timestamps:
    
    print("TIMESTAMP", timestamp)
    i = -1 
    for train_index, test_index in kf.split(stations):
        i = i + 1
        train_stations = stations[train_index]
        test_stations = stations[test_index]
        try:
            train_df = pd.concat([df.groupby('station_id').get_group(stn) for stn in train_stations])
            train_df = train_df.groupby('ts').get_group(timestamp)
            train_df.drop(columns = ['station_id', 'ts'])
            test_df = pd.concat([df.groupby('station_id').get_group(stn) for stn in test_stations])
            test_df = test_df.groupby('ts').get_group(timestamp)
            test_df.drop(columns = ['station_id', 'ts'])

        except KeyError:
            print("FAIL")
            continue
        
#         idw = Idw()
#         krig = Kriging()
# #         knn = KNeighborsRegressor(n_neighbors=6)
        
        X_train = np.array( train_df[['longitude', 'latitude']] )
        y_train = np.array( train_df[['PM2.5']] )
        X_test = np.array( test_df[['longitude', 'latitude']] )
        y_test = np.array( test_df[['PM2.5']] )
        
#         idw._fit(X_train, y_train)
#         krig.fit(X_train, y_train)
#         knn.fit(X_train, y_train)

        

        
        try:
#             idw_res = idw._predict(X_test)
#             krig_res = krig.predict(X_test)
# #             knn_res = knn.predict(X_test)
        
#             idw_rms_error = np.sqrt(mean_squared_error(idw_res, y_test))
#             krig_rms_error = np.sqrt(mean_squared_error(krig_res, y_test))
#             idw_mae_error = mean_absolute_error(idw_res, y_test)
#             krig_mae_error = mean_absolute_error(krig_res, y_test)
#             knn_rms_error = np.sqrt(mean_squared_error(knn_res, y_test))
#             knn_mae_error = mean_absolute_error(knn_res, y_test)
            
            

            tf.reset_default_graph()
            graph = tf.get_default_graph()
            gpflow.reset_default_session(graph=graph)


            xy_matern_1 = gpflow.kernels.Matern52(input_dim=2, ARD=True, active_dims=[0, 1])
#             xy_matern_2 = gpflow.kernels.Matern52(input_dim=2, ARD=True, active_dims=[0, 1])
            xy_rbf_3 = gpflow.kernels.RBF(input_dim = 2, ARD=True, active_dims=[0, 1])


#             kernel = gpflow.kernels.Linear(input_dim = 2, active_dims = [0, 1], ARD = True)
            kernel = xy_matern_1 +  xy_rbf_3

            model = gpflow.models.GPR(X_train, y_train, kern=kernel, mean_function=None)
            opt = gpflow.train.ScipyOptimizer()
            opt.minimize(model)

            mean, var = model.predict_y(X_test)
            
            gp_rms_error = np.sqrt(mean_squared_error(mean, y_test))
            gp_mae_error = mean_absolute_error(mean, y_test)

            
        except Exception as e:
            print(e)
            print("NO")
                  
            print(e)
            continue
            
        print("GP = ", gp_rms_error)
#         , "\nKrig = ", krig_rms_error, "\nIDW = ", idw_rms_error)
        
#         rmse['K'][timestamp][i] = krig_rms_error
#         rmse['I'][timestamp][i] = idw_rms_error
#         rmse['KNN'][timestamp][i] = knn_rms_error
        rmse['GP'][timestamp][i] = gp_rms_error
        
#         mae['K'][timestamp][i] = krig_mae_error
#         mae['I'][timestamp][i] = idw_mae_error
# #         mae['KNN'][timestamp][i] = knn_rms_error
        mae['GP'][timestamp][i] = gp_mae_error
        
        

W1102 12:04:07.115508 139634146854656 deprecation_wrapper.py:119] From /data/anaconda/envs/py36/lib/python3.6/site-packages/gpflow/session_manager.py:111: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1102 12:04:07.120035 139634146854656 deprecation_wrapper.py:119] From /data/anaconda/envs/py36/lib/python3.6/site-packages/gpflow/core/node.py:109: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1102 12:04:07.122555 139634146854656 deprecation_wrapper.py:119] From /data/anaconda/envs/py36/lib/python3.6/site-packages/gpflow/params/parameter.py:388: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



TIMESTAMP 0.0


W1102 12:04:08.036676 139634146854656 deprecation.py:323] From /data/anaconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1102 12:04:09.270968 139634146854656 deprecation.py:323] From /data/anaconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3744: setdiff1d (from tensorflow.python.ops.array_ops) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


GP =  14.097824699101043
GP =  14.019253883718205
GP =  11.458117378765515
GP =  10.827985670629452
GP =  11.572570058619991
GP =  17.18554610022084
TIMESTAMP 0.0027472527472527475
GP =  7.352718037891037
GP =  11.833181054092474
GP =  8.133455441802164
GP =  8.862396620323823
GP =  9.46914588187157
GP =  7.474769572827908
TIMESTAMP 0.005494505494505495
GP =  18.18133087989933
GP =  7.544905109010146
GP =  9.058601625285814
GP =  12.391834261420321
GP =  18.698931696135755
GP =  23.32808113240594
TIMESTAMP 0.008241758241758242
GP =  4.256586982900252
GP =  8.245328974076456
GP =  3.559585803723982
GP =  7.059781892788041
GP =  6.112305934557657
GP =  8.454994324578454
TIMESTAMP 0.01098901098901099
GP =  4.987256896038155
GP =  12.361513748875883
GP =  4.494809386389743
GP =  11.130826224516877
GP =  9.514111523294734
GP =  9.895745248919805
TIMESTAMP 0.013736263736263736
GP =  10.985189564842903
GP =  22.246057794264946
GP =  10.776593414638343
GP =  16.403259277612133
GP =  15.5770408

GP =  10.531842508861185
TIMESTAMP 0.13186813186813187
GP =  4.316096196583854
GP =  8.076245520067287
GP =  4.547002186540572
GP =  5.657912171197424
GP =  3.729785561886686
GP =  5.736524275831722
TIMESTAMP 0.1346153846153846
GP =  10.207556398638136
GP =  12.564875043311439
GP =  9.93417414596398
GP =  10.97617896368046
GP =  18.22865724595005
GP =  9.077762582871976
TIMESTAMP 0.13736263736263735
GP =  1.5102209846436243
GP =  4.072043006078602
GP =  2.8569489234359997
GP =  8.420718930327642
GP =  8.808207922083943
GP =  3.8344647788512014
TIMESTAMP 0.1401098901098901
GP =  4.396626236164723
GP =  10.765469999639372
GP =  6.763481065712022
GP =  5.480913213055654
GP =  6.383356720675259
GP =  5.588551130402371
TIMESTAMP 0.14285714285714285
GP =  3.8589312038914967
GP =  8.56493619787865
GP =  3.952150204419913
GP =  6.226337406657664
GP =  4.084170908072468
GP =  3.9664080639721293
TIMESTAMP 0.1456043956043956
GP =  2.805466843175705
GP =  10.907065862062511
GP =  4.690213175775062

...

GP =  8.738281862825463
GP =  8.851837914052068
GP =  7.309320372870323
TIMESTAMP 0.3956043956043956
GP =  4.290730206655694
GP =  7.501696193877586
GP =  8.22596048795024
GP =  9.714856159737915
GP =  6.02903508519053
GP =  6.814886221272859
TIMESTAMP 0.3983516483516483
GP =  4.82624156105626
GP =  6.849879954556942
GP =  8.854528536107424
GP =  6.518865243314094
GP =  5.731613522812025
GP =  7.255579972642326
TIMESTAMP 0.4010989010989011
GP =  4.7794535567197975
GP =  6.7121903973770305
GP =  5.5125533659406365
GP =  5.747479006703666
GP =  6.845610134466583
GP =  7.266444181838432
TIMESTAMP 0.40384615384615385
GP =  9.151947581287969
GP =  19.116680326502284
GP =  19.65802013947188
GP =  4.423522382595196
GP =  18.16235907959478
GP =  21.345186407053156
TIMESTAMP 0.4065934065934066
GP =  10.72886109231381
GP =  15.271009466898922
GP =  6.1894034128723545
GP =  11.00368740831152
GP =  7.751517714936796
GP =  5.3190901688593595
TIMESTAMP 0.4093406593406593
GP =  8.190800473133331
GP =

GP =  8.799809890788376
GP =  10.6385261957912
GP =  9.667342445758743
TIMESTAMP 0.5247252747252747
GP =  13.305216664122145
GP =  8.543003696042263
GP =  10.549757390795785
GP =  9.831191186310301
GP =  9.185665259642581
GP =  12.986876887962694
TIMESTAMP 0.5274725274725275
GP =  21.61374979803623
GP =  6.678660324810968
GP =  17.4773997184194
GP =  10.227707247908581
GP =  13.683660771063854
GP =  18.054420153399466
TIMESTAMP 0.5302197802197802
GP =  25.538596373748724
GP =  9.763376646811789
GP =  25.602983851574308
GP =  15.480047052631164
GP =  19.450746152738933
GP =  24.202120972731926
TIMESTAMP 0.532967032967033
GP =  11.205290505730462
GP =  36.54512589922899
GP =  10.769142007713397
GP =  14.626663854175105
GP =  12.801003465229368
GP =  30.069427797024527
TIMESTAMP 0.5357142857142857
GP =  1.1061870860868026
GP =  6.496079376329924
GP =  4.304508231392289
GP =  2.9579577885563326
GP =  3.136728177687552
GP =  6.071672605154847
TIMESTAMP 0.5384615384615384
GP =  9.96441591898

GP =  8.733137221197065
GP =  3.5445271159474716
GP =  15.612135994099752
GP =  52.712382724404456
TIMESTAMP 0.6538461538461539
GP =  24.15642843876792
GP =  29.301204723170915
GP =  21.159472281528636
GP =  10.011416013742773
GP =  16.580410770877155
GP =  39.95795128716738
TIMESTAMP 0.6565934065934066
GP =  30.847331990921123
GP =  36.365006222987915
GP =  36.34266859275899
GP =  14.271314442875878
GP =  52.27491983839098
GP =  32.91677163014886
TIMESTAMP 0.6593406593406593
GP =  31.13281892972847
GP =  31.903492204892082
GP =  58.89436043265405
GP =  23.92299554089343
GP =  36.17035451329203
GP =  42.32041394467359
TIMESTAMP 0.6620879120879121
GP =  52.10423703288315
GP =  28.130077187138987
GP =  62.55730979613032
GP =  42.69774907198677
GP =  64.994544691512
GP =  35.542541524745
TIMESTAMP 0.6648351648351648
GP =  23.540228339762077
GP =  69.9800753144469
GP =  33.48764701602342
GP =  39.22955555963122
GP =  64.04454042117504
GP =  67.23406382222899
TIMESTAMP 0.6675824175824175
GP

GP =  12.28536229504835
GP =  12.142332578929782
GP =  17.819627835542803
GP =  18.238757138948017
TIMESTAMP 0.782967032967033
GP =  20.5221499168261
GP =  49.802873780722074
GP =  34.381127160312815
GP =  16.16833085657012
GP =  35.08805585053876
GP =  30.01176241518654
TIMESTAMP 0.7857142857142857
GP =  11.05829459158748
GP =  10.538359674764486
GP =  22.64171203888222
GP =  9.539034631970196
GP =  9.246699414393325
GP =  18.20351601444916
TIMESTAMP 0.7884615384615384
GP =  9.017649562753057
GP =  15.813093568532171
GP =  7.019749330769641
GP =  4.781385234445173
GP =  7.036328436213922
GP =  19.18351236868048
TIMESTAMP 0.7912087912087912
GP =  16.13236412178615
GP =  42.05003722225104
GP =  11.730648991498297
GP =  9.397689010118745
GP =  15.140818772527853
GP =  11.234507900848335
TIMESTAMP 0.7939560439560439
GP =  48.81647083309322
GP =  36.956906518353925
GP =  32.58502336612193
GP =  28.846415091911865
GP =  47.57498428351281
GP =  30.689363660680236
TIMESTAMP 0.7967032967032966

GP =  14.971740869455031
GP =  6.400302454900373
GP =  9.891783399780826
GP =  11.454858501977194
GP =  4.935147971868192
GP =  17.53123851161565
TIMESTAMP 0.912087912087912
GP =  155.62564273348582
GP =  20.63650578606698
GP =  18.633321544226334
GP =  18.524849666381094
GP =  18.983879032163156
GP =  16.471757721717893
TIMESTAMP 0.9148351648351648
GP =  9.859190999307307
GP =  12.849591295582814
GP =  13.837571394750986
GP =  6.494770481859241
GP =  10.030905138223952
GP =  11.679379216634642
TIMESTAMP 0.9175824175824177
GP =  29.418251933010048
GP =  13.253238093154858
GP =  29.36898538093817
GP =  8.748212244965046
GP =  19.069746394085612
GP =  11.71230545759715
TIMESTAMP 0.9203296703296704
GP =  7.567356379720316
GP =  5.22864544403219
GP =  11.42542570021848
GP =  4.447541163914909
GP =  9.12380179059816
GP =  6.142314845364043
TIMESTAMP 0.9230769230769232
GP =  2.979112028194863
GP =  2.089252252598192
GP =  3.42188306394577
GP =  2.0035268964211412
GP =  2.5925220661694484
GP 

In [25]:
pd.DataFrame(rmse['GP']).mean(axis=0).mean()

15.586475195677046

In [ ]:
gp.mean(axis =0 ).mean()

In [ ]:
pd.DataFrame(rmse['K']).mean(axis =0).mean()

In [ ]:
gp.loc[:, (gp != 0).any(axis=0)].mean(axis = 0).mean()


In [ ]:
gp = pd.DataFrame(rmse['K'])
gp.loc[:, (gp != 0).any(axis=0)].mean(axis = 0).mean()


In [ ]:
from pykrige import OrdinaryKriging
OrdinaryKriging?

In [ ]:
gpflow.kernels.Linear?

In [ ]:
gpflow.kernels.Periodic?

In [ ]:
pd.DataFrame(rmse['K'])

In [ ]:
pd.DataFrame({'K':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps},'K1':{timestamp:{i:0 for i in range(6)} for timestamp in timestamps}})